In [1]:
#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.1//spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [557 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,184 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,012 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,384 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadcontent.net/

'/content/spark-3.4.1-bin-hadoop3'

In [2]:
from google.colab import drive

drive.mount("/content/gdrive")
!pwd
#put your own path in google drive
%cd "/content/gdrive/MyDrive/AI/Big_Data/archive_amazon"
!ls

Mounted at /content/gdrive
/content
/content/gdrive/MyDrive/AI/Big_Data/archive_amazon
amazon_review_polarity_csv.tgz	test.csv  train.csv


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('AmazonReviews').getOrCreate()

In [4]:
train_df = spark.read.csv('train.csv', header=False, inferSchema=True)
train_df = train_df.withColumnRenamed('_c0', 'polarity').withColumnRenamed('_c1', 'title').withColumnRenamed('_c2', 'text')

test_df = spark.read.csv('test.csv', header=False, inferSchema=True)
test_df = test_df.withColumnRenamed('_c0', 'polarity').withColumnRenamed('_c1', 'title').withColumnRenamed('_c2', 'text')

In [5]:
# polarity - 1 for negative and 2 for positive
train_df.show()

+--------+--------------------+--------------------+
|polarity|               title|                text|
+--------+--------------------+--------------------+
|       2|Stuning even for ...|This sound track ...|
|       2|The best soundtra...|I'm reading a lot...|
|       2|            Amazing!|"This soundtrack ...|
|       2|Excellent Soundtrack|I truly like this...|
|       2|Remember, Pull Yo...|If you've played ...|
|       2|an absolute maste...|I am quite sure a...|
|       1|        Buyer beware|"This is a self-p...|
|       2|      Glorious story|I loved Whisper o...|
|       2|    A FIVE STAR BOOK|I just finished r...|
|       2|Whispers of the W...|This was a easy t...|
|       1|          The Worst!|A complete waste ...|
|       2|          Great book|This was a great ...|
|       2|          Great Read|I thought this bo...|
|       1|           Oh please|I guess you have ...|
|       1|Awful beyond belief!|"I feel I have to...|
|       1|Don't try to fool...|It's glaringly 

In [6]:
train_df.printSchema()

root
 |-- polarity: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)



* polaridad es una columna entera que puede contener valores nulos. Este indica la polaridad del sentimiento (positivo/negativo) de cada opinión.
*  title y text son strings.

In [7]:
train_df.describe().show()
test_df.describe().show()


+-------+------------------+--------------------+--------------------+
|summary|          polarity|               title|                text|
+-------+------------------+--------------------+--------------------+
|  count|           3600000|             3599952|             3599987|
|   mean|               1.5|                 NaN|                null|
| stddev|0.5000000694444585|                 NaN|                null|
|    min|                 1|The Worst Thing ...|this is the best...|
|    max|                 2|         ♦ LOVE IT ♦|…were Marvin and ...|
+-------+------------------+--------------------+--------------------+

+-------+-----------------+--------------------+--------------------+
|summary|         polarity|               title|                text|
+-------+-----------------+--------------------+--------------------+
|  count|           400000|              399996|              399999|
|   mean|              1.5|6.600378819412851E28|                10.0|
| stddev|0

In [8]:
train_df = train_df.dropna()
test_df = test_df.dropna()


In [9]:
from pyspark.sql.functions import count

train_df.groupBy('polarity').agg(count('*').alias('count')).show()
test_df.groupBy('polarity').agg(count('*').alias('count')).show()

+--------+-------+
|polarity|  count|
+--------+-------+
|       1|1799969|
|       2|1799970|
+--------+-------+

+--------+------+
|polarity| count|
+--------+------+
|       1|199998|
|       2|199997|
+--------+------+



In [10]:
from pyspark.sql.functions import when

train_df = train_df.withColumn('label', when(train_df['polarity'] == 1, 0).otherwise(1))
test_df = test_df.withColumn('label', when(test_df['polarity'] == 1, 0).otherwise(1))

In [11]:
test_df.show()

+--------+--------------------+--------------------+-----+
|polarity|               title|                text|label|
+--------+--------------------+--------------------+-----+
|       2|            Great CD|"My lovely Pat ha...|    1|
|       2|One of the best g...|Despite the fact ...|    1|
|       1|Batteries died wi...|I bought this cha...|    0|
|       2|works fine, but M...|Check out Maha En...|    1|
|       2|Great for the non...|Reviewed quite a ...|    1|
|       1|DVD Player crappe...|I also began havi...|    0|
|       1|      Incorrect Disc|I love the style ...|    0|
|       1|DVD menu select p...|I cannot scroll t...|    0|
|       2|Unique Weird Orie...|"Exotic tales of ...|    1|
|       1|"Not an ""ultimat...|Firstly,I enjoyed...|    0|
|       2|Great book for tr...|I currently live ...|    1|
|       1|                Not!|If you want to li...|    0|
|       1|     A complete Bust|"This game requir...|    0|
|       2|TRULY MADE A DIFF...|I have been using...|    

In [12]:
#train_df = train_df.drop('polarity')
#test_df = test_df.drop('polarity')

In [13]:
# Processing the data

from pyspark.sql.functions import regexp_replace, lower, split

train_df = train_df.withColumn('title', regexp_replace(train_df['title'], '[^a-zA-Z0-9\\s]', ''))
train_df = train_df.withColumn('title', lower(train_df['title']))
#train_df = train_df.withColumn('title', split(train_df['title'], '\\s+'))

train_df = train_df.withColumn('text', regexp_replace(train_df['text'], '[^a-zA-Z0-9\\s]', ''))
train_df = train_df.withColumn('text', lower(train_df['text']))
#train_df = train_df.withColumn('text', split(train_df['text'], '\\s+'))

test_df = test_df.withColumn('title', regexp_replace(test_df['title'], '[^a-zA-Z0-9\\s]', ''))
test_df = test_df.withColumn('title', lower(test_df['title']))
#test_df = test_df.withColumn('title', split(test_df['title'], '\\s+'))

test_df = test_df.withColumn('text', regexp_replace(test_df['text'], '[^a-zA-Z0-9\\s]', ''))
test_df = test_df.withColumn('text', lower(test_df['text']))
#test_df = test_df.withColumn('text', split(test_df['text'], '\\s+'))

In [14]:
# Selecting only necessary columns

model_train_df = train_df.select('text', 'label')
model_test_df = test_df.select('text', 'label')

In [15]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

tokenizer = Tokenizer(inputCol='text', outputCol='words')

model_train_df = tokenizer.transform(model_train_df)
model_test_df = tokenizer.transform(model_test_df)

In [16]:
# we then convert the words column into an embed (feature vector)

hashingTF = HashingTF(inputCol='words', outputCol='rawFeatures')

model_train_df = hashingTF.transform(model_train_df)
model_test_df = hashingTF.transform(model_test_df)

In [17]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

idfModel = idf.fit(model_train_df)

train_df_lr = idfModel.transform(model_train_df)
test_df_lr = idfModel.transform(model_test_df)

In [18]:
lr = LogisticRegression(featuresCol='features', labelCol='label')

lrModel = lr.fit(train_df_lr)

In [19]:
predictions = lrModel.transform(test_df_lr)

In [20]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label')

auc = evaluator.evaluate(predictions)

print('AUC:', auc)

AUC: 0.9190643603961509
